In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install xlrd

In [4]:
coup_curr_ver = pd.read_csv("coups-merged.csv")
p5_df = pd.read_csv("p5v2018.csv")
poly_vio_df = pd.read_csv("MEPVv2018.csv")

p5_df.head(5)
print(poly_vio_df.columns)

Index(['scode', 'ccode', 'country', 'year', 'ind', 'intind', 'intviol',
       'intwar', 'civviol', 'civwar', 'ethviol', 'ethwar', 'inttot', 'civtot',
       'actotal', 'nborder', 'region', 'nregion'],
      dtype='object')


C:\Users\angle\AppData\Local\Temp\ipykernel_4208\1707638137.py:2: DtypeWarning: Columns (12,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  p5_df = pd.read_csv("p5v2018.csv")


In [5]:
#Rename columns from polity-5 dataset
#Dropped duplicate region column and old GDP colum
#Neighbors w/ violence vs. regional w/ violence (Do we need both?)
#


coup_curr_ver = coup_curr_ver.rename(columns={"INTIND": "indep_viol_mag", "INTVIOL":"intl_viol_mag", "INTWAR":"intl_war_mag", 
                        "CIVVIOL":"civil_viol_mag", "CIVWAR":"civil_war_mag", "ETHVIOL":"ethnic_viol_mag",
                        "ETHWAR":"ethnic_war_mag", "TOTINT":"total_intl_viol_mag", "TOTCIV":"total_civil_viol_mag",
                        "nINT": "neighbors_w_intl_viol", "nCIV":"neighbors_w_civil_viol", "nREGION":"neighbors_in_region",
                        "REGINT":"regional_intl_viol_mag", "REGCIV": "regional_civil_viol_mag", "REGAC":"total_regional_viol_mag",
                        "nRINT":"reg_states_intl_viol_count", "nRCIV":"reg_states_civil_viol_count"})

coup_curr_ver = coup_curr_ver.rename(columns={'democ': 'democracy', 'autoc': 'autocracy',
                        'polity2':'polity2_score', 'durable':'years_of_regime_pow', 'xrcomp':'exec_competitive_score', 
                        'xropen': 'exec_openness_score', 'xconst': 'exec_constraint_score', 'parreg' : 'political_regularity', 
                        'parcomp' : 'political_competitiveness','exrec': 'exec_recruitment'})

coup_curr_ver = coup_curr_ver.drop(columns=["REGION", "GDP per capita"])

print(coup_curr_ver.columns)

coup_curr_ver.to_csv("coup_merged_v2.csv")

Index(['Unnamed: 0.1', 'Unnamed: 0', 'coup_id', 'cowcode', 'country', 'year',
       'month', 'day', 'event_type', 'unrealized', 'realized', 'conspiracy',
       'attempt', 'military', 'dissident', 'rebel', 'palace', 'foreign',
       'auto', 'resign', 'popular', 'counter', 'other', 'noharm', 'injured',
       'killed', 'harrest', 'jailed', 'tried', 'fled', 'exile', 'region',
       'sub-region', 'gini', 'indep_viol_mag', 'intl_viol_mag', 'intl_war_mag',
       'civil_viol_mag', 'civil_war_mag', 'ethnic_viol_mag', 'ethnic_war_mag',
       'total_intl_viol_mag', 'total_civil_viol_mag', 'neighbors_w_intl_viol',
       'neighbors_w_civil_viol', 'neighbors_in_region',
       'regional_intl_viol_mag', 'regional_civil_viol_mag',
       'total_regional_viol_mag', 'reg_states_intl_viol_count',
       'reg_states_civil_viol_count', 'democracy', 'autocracy',
       'polity2_score', 'years_of_regime_pow', 'exec_competitive_score',
       'exec_openness_score', 'exec_constraint_score', 'political_

In [6]:
x = coup_curr_ver.isna().sum(axis=0)
x[x > 0]

'''Based on guide, for polity score: 
-Cases of -77 are to be treated as 0 (and perhaps we add another column),
-Cases of -88 are to have the differences spreas across them:

    For example, country X has a POLITY score of -7 in 1957, followed
    by three years of -88 and, finally, a score of +5 in 1961. The change
    (+12) would be prorated over the intervening three years at a rate of
    per year, so that the converted scores would be as follows: 1957 -7;
    1958 -4; 1959 -1; 1960 +2; and 1961 +5.

-Cases of -66 are true missingness (Page 17 of Polity5 Guide)
'''
country_ids = ['coup_id', 'cowcode', 'country', 'year']
polity_old_cols = ['democ', 'autoc', 'polity', 'polity2', 'durable', 'xrreg', 'xrcomp',
       'xropen', 'xconst', 'parreg', 'parcomp', 'exrec']
polity_imputation_cols = ['democracy', 'autocracy',
       'polity2_score', 'years_of_regime_pow', 'exec_competitive_score',
       'exec_openness_score', 'exec_constraint_score', 'political_regularity',
       'political_competitiveness', 'exec_recruitment']

coup_pol_impute = coup_curr_ver[country_ids + polity_imputation_cols]


# Impute 0s for -77 and create interregum (b/t governments) column 
display(coup_pol_impute.head(5))

coup_curr_ver["standing_regime"] = np.where(coup_curr_ver["democracy"] == -77, 0, 1)
for feature in polity_imputation_cols:
    coup_curr_ver[feature].mask(coup_curr_ver[feature] == -77, 0, inplace=True)
display(coup_curr_ver)

#Impute 

coup_curr_ver.to_csv("-77_yeeted.csv")



interregum_coup = coup_pol_impute[coup_pol_impute['democracy'] == -88]
display(interregum_coup)

interregum_coup['country'].value_counts()


coup_id  cowcode       country  year  democracy  autocracy  \
0  43910302014      439  Burkina Faso  2014        2.0        2.0   
1  51605132015      516       Burundi  2015        2.0        3.0   
2  48304132006      483          Chad  2006        1.0        3.0   
3  65102112011      651         Egypt  2011        0.0        2.0   
4  65107032013      651         Egypt  2013        0.0        4.0   

   polity2_score  years_of_regime_pow  exec_competitive_score  \
0            0.0                 13.0                     0.0   
1           -1.0                  0.0                     1.0   
2           -2.0                 14.0                     0.0   
3           -2.0                  6.0                     0.0   
4           -4.0                  0.0                     0.0   

   exec_openness_score  exec_constraint_score  political_regularity  \
0                  0.0                    3.0                   3.0   
1                  4.0                    4.0                   3.0   
2                  0.0                    2.0                   3.0   
3                  0.0                    2.0                   2.0   
4                  0.0                    3.0                   4.0   

   political_competitiveness  exec_recruitment  
0                        4.0               5.0  
1                        3.0               3.0  
2                        3.0               5.0  
3                        0.0               5.0  
4                        2.0               5.0

Unnamed: 0.1  Unnamed: 0      coup_id  cowcode       country  year  \
0               0           0  43910302014      439  Burkina Faso  2014   
1               1           1  51605132015      516       Burundi  2015   
2               2           2  48304132006      483          Chad  2006   
3               3           3  65102112011      651         Egypt  2011   
4               4           4  65107032013      651         Egypt  2013   
..            ...         ...          ...      ...           ...   ...   
938           938         938  55106141981      551        Zambia  1981   
939           939         939  55110071988      551        Zambia  1988   
940           940         940  55106301990      551        Zambia  1990   
941           941         941  55110281997      551        Zambia  1997   
942           942         942  55201111999      552      Zimbabwe  1999   

     month  day  event_type  unrealized  ...    gdppc  gdpgrwth  cntry_change  \
0       10   30        coup           0  ...   1930.0      8.06             0   
1        5   13   attempted           1  ...    795.0      3.91             0   
2        4   13   attempted           1  ...   1570.0     0.528             0   
3        2   11        coup           0  ...  10300.0      1.33             0   
4        7    3        coup           0  ...  10400.0      1.79             0   
..     ...  ...         ...         ...  ...      ...       ...           ...   
938      6   14  conspiracy           1  ...   2180.0      2.48             0   
939     10    7  conspiracy           1  ...   2050.0     0.955             0   
940      6   30   attempted           1  ...   2130.0     −1.96             0   
941     10   28   attempted           1  ...   1930.0     −1.86             0   
942      1   11  conspiracy           1  ...   4350.0     −3.17             0   

     pre_coup  pre_coup5  pre_coup_fail  pre_coup_succ  pre_leader_harm  \
0           7          0              1              6                1   
1          14          0              6              8                2   
2          10          0              6              4                1   
3          11          0              7              4                0   
4          12          1              7              5                0   
..        ...        ...            ...            ...              ...   
938         1          1              1              0                0   
939         2          0              2              0                0   
940         3          1              3              0                0   
941         4          0              4              0                0   
942         0          0              0              0                0   

     pre_leader_court  standing_regime  
0                   2                1  
1                   0                1  
2                   0                1  
3                   1                1  
4                   3                1  
..                ...              ...  
938                 0                1  
939                 0                1  
940                 0                1  
941                 0                1  
942                 0                1  

[943 rows x 76 columns]

coup_id  cowcode     country  year  democracy  autocracy  \
39   70304082010      703  Kyrgyzstan  2010      -88.0      -88.0   
96   16006161955      160   Argentina  1955      -88.0      -88.0   
97   16009161955      160   Argentina  1955      -88.0      -88.0   
98   16011131955      160   Argentina  1955      -88.0      -88.0   
99   16006091956      160   Argentina  1956      -88.0      -88.0   
..           ...      ...         ...   ...        ...        ...   
852  80010141973      800    Thailand  1973      -88.0      -88.0   
854  80003261977      800    Thailand  1977      -88.0      -88.0   
855  80010201977      800    Thailand  1977      -88.0      -88.0   
881  64005271960      640      Turkey  1960      -88.0      -88.0   
888  50004151966      500      Uganda  1966      -88.0      -88.0   

     polity2_score  years_of_regime_pow  exec_competitive_score  \
39             4.0                  0.0                   -88.0   
96            -6.0                  0.0                   -88.0   
97            -6.0                  0.0                   -88.0   
98            -6.0                  0.0                   -88.0   
99            -2.0                  0.0                   -88.0   
..             ...                  ...                     ...   
852           -2.0                  0.0                   -88.0   
854           -2.0                  0.0                   -88.0   
855           -2.0                  0.0                   -88.0   
881            7.0                  0.0                   -88.0   
888            0.0                  0.0                   -88.0   

     exec_openness_score  exec_constraint_score  political_regularity  \
39                 -88.0                  -88.0                 -88.0   
96                 -88.0                  -88.0                 -88.0   
97                 -88.0                  -88.0                 -88.0   
98                 -88.0                  -88.0                 -88.0   
99                 -88.0                  -88.0                 -88.0   
..                   ...                    ...                   ...   
852                -88.0                  -88.0                 -88.0   
854                -88.0                  -88.0                 -88.0   
855                -88.0                  -88.0                 -88.0   
881                -88.0                  -88.0                 -88.0   
888                -88.0                  -88.0                 -88.0   

     political_competitiveness  exec_recruitment  
39                       -88.0             -88.0  
96                       -88.0             -88.0  
97                       -88.0             -88.0  
98                       -88.0             -88.0  
99                       -88.0             -88.0  
..                         ...               ...  
852                      -88.0             -88.0  
854                      -88.0             -88.0  
855                      -88.0             -88.0  
881                      -88.0             -88.0  
888                      -88.0             -88.0  

[61 rows x 14 columns]

Syria                               7
Portugal                            6
Argentina                           5
Thailand                            3
Dominican Republic                  3
Philippines                         3
Haiti                               2
El Salvador                         2
Hungary                             2
German Democratic Republic          2
Fiji                                2
Mali                                2
Democratic Republic of the Congo    2
Cuba                                2
Burundi                             2
Brazil                              2
Senegal                             1
Rumania                             1
Turkey                              1
Sudan                               1
Kyrgyzstan                          1
Peru                                1
Liberia                             1
Italy                               1
Guinea-Bissau                       1
Ecuador                             1
Cyprus      

In [7]:
# Impute 0s for -66 and create interregum (b/t governments) column 

coup_curr_ver["standing_regime"].mask(coup_curr_ver['democracy'] == -66, 0, inplace=True)
for feature in polity_imputation_cols:
    coup_curr_ver[feature].mask(coup_curr_ver[feature] == -66, 0, inplace=True)
#display(coup_curr_ver)

coup_curr_ver.to_csv("-77_yeeted.csv")


#Impute value from nearest neighbor for missing value at row with -88 (rationale: -88 indicates a transition period)

relevant_p5 = p5_df[["country"] + polity_old_cols]

coup_curr_ver["standing_regime"].mask(coup_curr_ver['democracy'] == -88, 0, inplace=True)

country_list = [x for x in coup_curr_ver['country'].unique() if x in relevant_p5['country'].unique()]

for country in country_list:
    country_df = relevant_p5[relevant_p5['country'] == country]

    for i in range(len(polity_imputation_cols)):
        feature_avg = int(np.round(np.mean(country_df[country_df[polity_old_cols[i]] != -88][polity_old_cols[i]])))
        
        # Impute means of non-standing government and missing observation
        coup_curr_ver[polity_imputation_cols[i]].mask(coup_curr_ver[polity_imputation_cols[i]] == -88, feature_avg, inplace=True)

        coup_curr_ver[polity_imputation_cols[i]].mask(coup_curr_ver[polity_imputation_cols[i]].isna(), feature_avg, inplace=True)

coup_curr_ver.to_csv("coup_merged_v3.csv")

y = coup_curr_ver.isna().sum(axis=0)
display(coup_curr_ver.isna().sum(axis=0))
y[y > 0]       








TypeError: unsupported operand type(s) for +: 'float' and 'str'